In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd

# set the API endpoint and parameters
url = "https://api.coingecko.com/api/v3/coins/ripple/market_chart"
params = {
    "vs_currency": "usd",
    "days": "2053",
    "interval": "1d"
}

# make a GET request to the API to get the tether market chart data
response = requests.get(url, params=params)

# extract the market chart data from the response
market_chart_data = response.json()

# create a dataframe to store the market chart data
df = pd.DataFrame(market_chart_data["prices"], columns=["date", "price"])

# convert the Unix timestamps in the "date" column to datetime objects
df["date"] = pd.to_datetime(df["date"],unit="ms")

# add new columns for open, high, low, and close prices
df["open"] = df["price"].shift(1)
df["high"] = df[["price", "open"]].max(axis=1)
df["low"] = df[["price", "open"]].min(axis=1)
df["close"] = df["price"]

# set the index of the dataframe "date" column
df.set_index("date", inplace=True)

# drop the "price" column
df.drop("price", axis=1, inplace=True)

# set the column order
df = df[["open", "high", "low", "close"]]

# add new columns for volume and market cap data
df["volume"] = [v[1] for v in market_chart_data["total_volumes"]]
df["marketCap"] = [mc[1] for mc in market_chart_data["market_caps"]]

# print the dataframe
print(df)



                         open      high       low     close        volume  \
date                                                                        
2017-08-02 00:00:00       NaN  0.173609  0.173609  0.173609  5.893240e+07   
2017-08-03 00:00:00  0.173609  0.174492  0.173609  0.174492  3.155220e+07   
2017-08-04 00:00:00  0.174492  0.174492  0.174084  0.174084  4.533110e+07   
2017-08-05 00:00:00  0.174084  0.184159  0.174084  0.184159  1.126470e+08   
2017-08-06 00:00:00  0.184159  0.184159  0.180304  0.180304  7.460280e+07   
...                       ...       ...       ...       ...           ...   
2023-03-13 00:00:00  0.366037  0.372470  0.366037  0.372470  1.204499e+09   
2023-03-14 00:00:00  0.372470  0.374133  0.372470  0.374133  1.880604e+09   
2023-03-15 00:00:00  0.374133  0.374893  0.374133  0.374893  1.673162e+09   
2023-03-16 00:00:00  0.374893  0.374893  0.362040  0.362040  1.224434e+09   
2023-03-16 18:09:08  0.362040  0.366415  0.362040  0.366415  8.525170e+08   

In [2]:
df

,open,high,low,close,volume,marketCap
date,,,,,,
2017-08-02 00:00:00,NaN,0.173609,0.173609,0.173609,5.893240e+07,6.654970e+09
2017-08-03 00:00:00,0.173609,0.174492,0.173609,0.174492,3.155220e+07,6.690694e+09
2017-08-04 00:00:00,0.174492,0.174492,0.174084,0.174084,4.533110e+07,6.675049e+09
2017-08-05 00:00:00,0.174084,0.184159,0.174084,0.184159,1.126470e+08,7.061364e+09
2017-08-06 00:00:00,0.184159,0.184159,0.180304,0.180304,7.460280e+07,6.913548e+09
...,...,...,...,...,...,...
2023-03-13 00:00:00,0.366037,0.372470,0.366037,0.372470,1.204499e+09,1.891087e+10
2023-03-14 00:00:00,0.372470,0.374133,0.372470,0.374133,1.880604e+09,1.906742e+10
2023-03-15 00:00:00,0.374133,0.374893,0.374133,0.374893,1.673162e+09,1.909549e+10


In [3]:
#Checking the dtype of the index
date_index = df.index
print(date_index)

DatetimeIndex(['2017-08-02 00:00:00', '2017-08-03 00:00:00',
               '2017-08-04 00:00:00', '2017-08-05 00:00:00',
               '2017-08-06 00:00:00', '2017-08-07 00:00:00',
               '2017-08-08 00:00:00', '2017-08-09 00:00:00',
               '2017-08-10 00:00:00', '2017-08-11 00:00:00',
               ...
               '2023-03-08 00:00:00', '2023-03-09 00:00:00',
               '2023-03-10 00:00:00', '2023-03-11 00:00:00',
               '2023-03-12 00:00:00', '2023-03-13 00:00:00',
               '2023-03-14 00:00:00', '2023-03-15 00:00:00',
               '2023-03-16 00:00:00', '2023-03-16 18:09:08'],
              dtype='datetime64[ns]', name='date', length=2054, freq=None)


Group the rows by the date portion of the timestamp, and select only the row with the earliest timestamp for each date using the first() method. 
The resulting DataFrame will have only one row per date, with the earliest timestamp of the day and the corresponding value.



In [4]:
df_grouped_ripple = df.groupby(df.index.date).first()
df_grouped_ripple

,open,high,low,close,volume,marketCap
2017-08-02,NaN,0.173609,0.173609,0.173609,5.893240e+07,6.654970e+09
2017-08-03,0.173609,0.174492,0.173609,0.174492,3.155220e+07,6.690694e+09
2017-08-04,0.174492,0.174492,0.174084,0.174084,4.533110e+07,6.675049e+09
2017-08-05,0.174084,0.184159,0.174084,0.184159,1.126470e+08,7.061364e+09
2017-08-06,0.184159,0.184159,0.180304,0.180304,7.460280e+07,6.913548e+09
...,...,...,...,...,...,...
2023-03-12,0.371984,0.371984,0.366037,0.366037,1.134168e+09,1.861825e+10
2023-03-13,0.366037,0.372470,0.366037,0.372470,1.204499e+09,1.891087e+10
2023-03-14,0.372470,0.374133,0.372470,0.374133,1.880604e+09,1.906742e+10
2023-03-15,0.374133,0.374893,0.374133,0.374893,1.673162e+09,1.909549e+10


In [5]:
# Check if the index is unique
print(df_grouped_ripple.index.is_unique)

True


In [6]:
#Cleaning Data
df_grouped_ripple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2053 entries, 2017-08-02 to 2023-03-16
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       2052 non-null   float64
 1   high       2053 non-null   float64
 2   low        2053 non-null   float64
 3   close      2053 non-null   float64
 4   volume     2053 non-null   float64
 5   marketCap  2053 non-null   float64
dtypes: float64(6)
memory usage: 176.8+ KB


The Dtype are the correct float64 for the 6 columns and the index 'date', dtype='datetime64[ns].
we checked that we don't have any repeat date already.

In [7]:
# Check for missing values
if df_grouped_ripple.isnull().values.any():
    print("The dataframe contains missing values.")
else:
    print("The dataframe does not contain any missing values.")

The dataframe contains missing values.


In [8]:
# drop rows with any missing value
df_grouped_ripple.dropna(inplace=True)
df_grouped_ripple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2052 entries, 2017-08-03 to 2023-03-16
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       2052 non-null   float64
 1   high       2052 non-null   float64
 2   low        2052 non-null   float64
 3   close      2052 non-null   float64
 4   volume     2052 non-null   float64
 5   marketCap  2052 non-null   float64
dtypes: float64(6)
memory usage: 112.2+ KB


In [9]:
df_grouped_ripple

,open,high,low,close,volume,marketCap
2017-08-03,0.173609,0.174492,0.173609,0.174492,3.155220e+07,6.690694e+09
2017-08-04,0.174492,0.174492,0.174084,0.174084,4.533110e+07,6.675049e+09
2017-08-05,0.174084,0.184159,0.174084,0.184159,1.126470e+08,7.061364e+09
2017-08-06,0.184159,0.184159,0.180304,0.180304,7.460280e+07,6.913548e+09
2017-08-07,0.180304,0.180304,0.178999,0.178999,5.867390e+07,6.863509e+09
...,...,...,...,...,...,...
2023-03-12,0.371984,0.371984,0.366037,0.366037,1.134168e+09,1.861825e+10
2023-03-13,0.366037,0.372470,0.366037,0.372470,1.204499e+09,1.891087e+10
2023-03-14,0.372470,0.374133,0.372470,0.374133,1.880604e+09,1.906742e+10
2023-03-15,0.374133,0.374893,0.374133,0.374893,1.673162e+09,1.909549e+10
